# Your Title Here

**Name(s)**: Landon Marchetti and Lyn Mansfield

**Website Link**: (your website link)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from matplotlib.backends.backend_pdf import PdfPages

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from dsc80_utils import * # Feel free to uncomment and use this.

In [2]:
%load_ext autoreload
%autoreload 2

## Step 1: Introduction

# TODO

Which legends are the most meta-defining? Difference in win rate between when it's banned vs when it's not

How important is the first kill of a game? 
- not important at all actually lol, the winrates are all the same no matter what

Which characters are the most banned? Which characters are the best?

What archetype of character's performance matters the most?

How important is early success?

## Step 2: Data Cleaning and Exploratory Data Analysis

What are "partial" rows missing from their data? <== me!

In [195]:
# TODO

from pathlib import Path

dataset_path = Path("./data/2025_LoL_clean_stats.csv")
# /data/2025_LoL_clean_stats.csv
data = pd.read_csv(dataset_path)
data

,Unnamed: 0,gameid,datacompleteness,url,...,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm
0,0,LOLTMNT03_179647,complete,NaN,...,0,-1,-1,8.82
1,1,LOLTMNT03_179647,complete,NaN,...,132,-1,-1,5.39
2,2,LOLTMNT03_179647,complete,NaN,...,0,-1,-1,7.88
...,...,...,...,...,...,...,...,...,...
60177,60177,LOLTMNT06_120342,complete,NaN,...,0,-1,-1,1.10
60178,60178,LOLTMNT06_120342,complete,NaN,...,159,-1,-1,34.30
60179,60179,LOLTMNT06_120342,complete,NaN,...,222,-1,-1,36.61


In [182]:
# for boolean columns, give special treatment
boolean_columns = ['firstblood', 'firstbloodkill', 'firstbloodassist', 'firstbloodvictim']
def transform_int_to_bool(x):
    if x == 1:
        return True
    if x == 0:
        return False
for column in boolean_columns:
    data[column] = data[column].apply(lambda x: transform_int_to_bool(x) if x >= 0 else np.nan)

# clearly label if players won or lost
data['result'] = data['result'].apply(lambda x: 'Won' if x == 1 else 'Lost')

data

,gameid,datacompleteness,url,league,...,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm
0,LOLTMNT03_179647,complete,NaN,LFL2,...,0,-1,-1,8.82
1,LOLTMNT03_179647,complete,NaN,LFL2,...,132,-1,-1,5.39
2,LOLTMNT03_179647,complete,NaN,LFL2,...,0,-1,-1,7.88
...,...,...,...,...,...,...,...,...,...
60177,LOLTMNT06_120342,complete,NaN,CD,...,0,-1,-1,1.10
60178,LOLTMNT06_120342,complete,NaN,CD,...,159,-1,-1,34.30
60179,LOLTMNT06_120342,complete,NaN,CD,...,222,-1,-1,36.61


In [186]:
data.to_csv("2025_LoL_clean_stats.csv")

In [188]:
type_frequencies = data.dtypes.value_counts()
type_frequencies.index = type_frequencies.index.map(str)
fig = px.bar(type_frequencies, 
	labels={'index': 'Data type', 'value': 'Frequency'}
	)
fig.update_layout(
    title={
        'text': 'Frequencies of Relevant Data Types',
        'font': {'size': 24}
    }
)
fig.show()

In [187]:
num_missing = data.apply(pd.isna).sum()
num_missing = num_missing / data.shape[0] * 100

fig = px.bar(num_missing[num_missing != 0].sort_values(), 
	labels={'index': 'Column name', 'value': '% missing'},
	title='Data missing from partial rows'
	)
fig.show()

## Step 3: Assessment of Missingness

In [124]:
# TODO

def univariate_plots(df, title):
    pdf_pages = PdfPages(f'univariate_plots_{title}.pdf')

    plt.figure()

    for column in df.columns:
        fig, ax = plt.subplots()
        ax.hist(df[column], bins=200)
        ax.set_title(f'Graph of {column}')
        pdf_pages.savefig(fig)
        plt.close(fig)
    
    pdf_pages.close()

def bivariate_plot(df, categorical_column, target_column, title):
    pdf_pages = PdfPage(f'bivariate_plots_{title}.pdf')

    plt.figure()
    
    for column in df.columns:
        fig, ax = plt.subplots()
        ax.hist(df[column])
        ax.set_title(f'Graph of {column}')
        pdf_pages.savefig(fig)
        plt.close(fig)
    
    pdf_pages.close()

In [150]:
player_data = data[data['damageshare'].notna()]

In [151]:
fig = player_data['dpm'].hist()
fig.show()

In [152]:
fig = player_data['damagetakenperminute'].hist()
fig.show()

In [154]:
fig = player_data['damagemitigatedperminute'].hist()
fig.show()

## Step 4: Hypothesis Testing

In [224]:
# TODO

player_rows = data[data['playername'].notna()]
team_rows = data[data['playername'].isna()]

In [292]:
player_rows['result'] = player_rows['result'].apply(lambda x: 1 if x == "Won" else 0)
champion_win_rates = player_rows.groupby("champion")['result'].mean()
champion_win_rates = champion_win_rates.rename("Win rate")
champion_win_rates

champion
Aatrox    0.51
Ahri      0.49
Akali     0.51
          ... 
Zilean    0.56
Zoe       0.53
Zyra      0.52
Name: Win rate, Length: 168, dtype: float64

In [293]:
times_picked = player_rows['champion'].value_counts()
times_picked = times_picked.rename('Times picked')

In [294]:
ban_columns = team_rows[['ban1', 'ban2', 'ban3', 'ban4', 'ban5']]
bans_series = ban_columns.stack()
champion_ban_rates = bans_series.value_counts() / bans_series.shape[0] * 5 
# multiply by 5 because each team bans 5 champions, so there are (ban_series.shape[0] / 5) games total
champion_ban_rates = champion_ban_rates.rename('Ban rate')
champion_ban_rates

Kalista    2.28e-01
Vi         2.03e-01
Varus      1.95e-01
             ...   
Sona       1.00e-04
Nasus      1.00e-04
Illaoi     1.00e-04
Name: Ban rate, Length: 168, dtype: float64

In [298]:
merged_df = pd.DataFrame(champion_win_rates)
merged_df = merged_df.merge(champion_ban_rates, left_index=True, right_index=True)
merged_df = merged_df.merge(times_picked, left_index=True, right_index=True)

merged_df = merged_df.reset_index().rename(columns={'index': 'Champion name'})
merged_df

,Champion name,Win rate,Ban rate,Times picked
0,Aatrox,0.51,1.07e-02,396
1,Ahri,0.49,5.18e-02,903
2,Akali,0.51,4.20e-02,362
...,...,...,...,...
164,Zilean,0.56,3.00e-04,16
165,Zoe,0.53,6.90e-03,36
166,Zyra,0.52,9.40e-02,370


In [299]:
from sklearn.linear_model import LinearRegression
from scipy import stats

# Reshape X for sklearn (requires 2D array)
X = merged_df[['Ban rate']]
y = merged_df['Win rate']

# Fit model
model = LinearRegression().fit(X, y)

# Calculate p-value manually
slope = model.coef_[0]
intercept = model.intercept_
predictions = model.predict(X)

# Residuals and standard error
residuals = y - predictions
dof = len(X) - 2  # Degrees of freedom
mse = np.sum(residuals**2) / dof
se_slope = np.sqrt(mse / np.sum((X['Ban rate'] - X['Ban rate'].mean())**2))
t_stat = slope / se_slope
p_value = 2 * stats.t.sf(np.abs(t_stat), dof)
p_value
# Lyn: P-value is pretty low

np.float64(0.2569915129656933)

In [300]:
fig = px.scatter(merged_df, x='Ban rate', y='Win rate', hover_data='Champion name')
fig.add_hline(
    y=0.5,
    line_dash="dash",  # Makes the line dashed
    line_color="black",  # Adjust color as needed
    line_width=2,
    opacity=0.2,
)

# Add linear regression trace
fig.add_trace(
    go.Scatter(
        x=merged_df['Ban rate'],
        y=model.predict(merged_df[['Ban rate']]),
        mode='lines',
        name='Regression Line',
        opacity=0.3,
        line=dict(color='red')
))

fig.show()
# Lyn: Looks like there's a lot of outlier data from champions that nobody picks or bothers banning

In [336]:
#merged_df.sort_values(by='Times picked').plot.bar(x='Champion name', y='Times picked')
# Filter out rarely picked champions to see if our model improves
print(sifted_df['Times picked'].sum() / 1000)
# Lyn: Let's try keeping only champions that are picked one time in every thousand, that seems fair
sifted_df = merged_df.query("`Times picked` > 50")
sifted_df.shape[0] / merged_df.shape[0]

48.949


0.6167664670658682

In [334]:
from sklearn.linear_model import LinearRegression
from scipy import stats

# Reshape X for sklearn (requires 2D array)
X = sifted_df[['Ban rate']]
y = sifted_df['Win rate']

# Fit model
model = LinearRegression().fit(X, y)

# Calculate p-value manually
slope = model.coef_[0]
intercept = model.intercept_
predictions = model.predict(X)

# Residuals and standard error
residuals = y - predictions
dof = len(X) - 2  # Degrees of freedom
mse = np.sum(residuals**2) / dof
se_slope = np.sqrt(mse / np.sum((X['Ban rate'] - X['Ban rate'].mean())**2))
t_stat = slope / se_slope
p_value = 2 * stats.t.sf(np.abs(t_stat), dof)
p_value
# Lyn: Much better p-value! Looks like we can conclude that ban rates are correlated with win rate.

np.float64(0.024004906650771665)

In [335]:
fig = px.scatter(sifted_df, x='Ban rate', y='Win rate', hover_data='Champion name')
fig.add_hline(
    y=0.5,
    line_dash="dash",  # Makes the line dashed
    line_color="black",  # Adjust color as needed
    line_width=2,
    opacity=0.2,
)

# Add linear regression trace
fig.add_trace(
    go.Scatter(
        x=sifted_df['Ban rate'],
        y=model.predict(sifted_df[['Ban rate']]),
        mode='lines',
        name='Regression Line',
        opacity=0.3,
        line=dict(color='red')
))

fig.show()

## Step 5: Framing a Prediction Problem

In [ ]:
# TODO
Use one-hot encoding!

## Step 6: Baseline Model

In [ ]:
# TODO

## Step 7: Final Model

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO